# Detecting BDM In Superbowl Commercials

In [33]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
%pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
BDM_excel = pd.read_excel(f'{os.getenv("BDM_EXCEL_FILE")}')
final_excel = pd.read_excel(f'{os.getenv("FINAL_EXCEL_FILE")}')

In [35]:
BDM_excel = pd.read_excel(f'{os.getenv("BDM_EXCEL_FILE")}')
final_excel = pd.read_excel(f'{os.getenv("FINAL_EXCEL_FILE")}')



In [36]:
final_excel = final_excel.merge(
    BDM_excel[['AdNumber', 'BDM']], 
    on='AdNumber', 
    how='left',
    suffixes=('_old', '')
).drop('BDM_old', axis=1, errors='ignore')


# print number of rows where BDM is NaN, 0 and 1
print(f"Number of rows where BDM is NaN: {final_excel[final_excel['BDM'].isna()].shape[0]}")
print(f"Number of rows where BDM is 0: {final_excel[final_excel['BDM'] == 0].shape[0]}")
print(f"Number of rows where BDM is 1: {final_excel[final_excel['BDM'] == 1].shape[0]}")


Number of rows where BDM is NaN: 22
Number of rows where BDM is 0: 411
Number of rows where BDM is 1: 139


In [37]:
ad_df = final_excel.groupby(['cont_primary_product_type', 'BRAND', 'AdNumber', "BDM"]).size().reset_index(name='count')
ad_df.rename(columns={'cont_primary_product_type': 'product_category', 'BRAND': 'brand', 'AdNumber': 'commercial_number'}, inplace=True)
ad_df.drop(columns=['count'], inplace=True)
ad_df.head(10)

,product_category,brand,commercial_number,BDM
0,1.0,AvocadosfromMexico,AD0357,0.0
1,1.0,AvocadosfromMexico,AD0414,1.0
2,1.0,AvocadosfromMexico,AD0474,0.0
3,1.0,AvocadosfromMexico,AD0525,1.0
4,1.0,AvocadosfromMexico,AD0584,1.0
5,1.0,AvocadosfromMexico,AD0635,1.0
6,1.0,AvocadosfromMexico,AD0745,0.0
7,1.0,BUBLY,AD0586,1.0
8,1.0,Bai,AD0475,0.0
9,1.0,Butterfinger,AD0420,1.0


## Retrieving Transcript

In [38]:
import glob
from pathlib import Path

# TODO: Implement this so that you can reuse it in the streamlit app

# Get all txt files recursively from ADS_DIR
ads_dir = Path(os.getenv("ADS_DIR"))
transcript_files = glob.glob(str(ads_dir / "**/*.txt"), recursive=True)
# print transcript_files
print(transcript_files)
# Create a dictionary mapping commercial numbers to file paths
transcript_map = {Path(f).stem: f for f in transcript_files}

# Update transcripts in dataframe
ad_df['transcript'] = ''
for idx, row in ad_df.iterrows():
    commercial_num = row['commercial_number']
    if commercial_num in transcript_map:
        try:
            with open(transcript_map[commercial_num], 'r', encoding='utf-8') as f:
                ad_df.at[idx, 'transcript'] = f.read().strip()
        except FileNotFoundError:
            ad_df.at[idx, 'transcript'] = None
    else:
        ad_df.at[idx, 'transcript'] = None

ad_df[ad_df['transcript'].notna()]
ad_df["audio_only_transcript"] = ad_df["transcript"]
ad_df.head(10)

['/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0536.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0537.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0535.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0534.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0524.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0530.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0531.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0525.txt', '/Users/nilst/Development/Commercial-Brand-Differentiating-Message-Analysis/ADs/ADs_IG_2018/AD0527.txt', '/Users/nilst/Development/Commercial-Brand-Differentia

,product_category,brand,commercial_number,BDM,transcript,audio_only_transcript
0,1.0,AvocadosfromMexico,AD0357,0.0,"With the next pick in the first draft ever, Au...","With the next pick in the first draft ever, Au..."
1,1.0,AvocadosfromMexico,AD0414,1.0,Over here we have the cube of Rubik. This simp...,Over here we have the cube of Rubik. This simp...
2,1.0,AvocadosfromMexico,AD0474,0.0,"Guys, how can we be a secret society if we can...","Guys, how can we be a secret society if we can..."
3,1.0,AvocadosfromMexico,AD0525,1.0,"Well, we did it. We sealed the bad out and kep...","Well, we did it. We sealed the bad out and kep..."
4,1.0,AvocadosfromMexico,AD0584,1.0,"Pageantry, poise, welcome to the dog show wher...","Pageantry, poise, welcome to the dog show wher..."
5,1.0,AvocadosfromMexico,AD0635,1.0,Welcome to the Avocados from Mexico shopping n...,Welcome to the Avocados from Mexico shopping n...
6,1.0,AvocadosfromMexico,AD0745,0.0,This is officially the worst tailgate I've eve...,This is officially the worst tailgate I've eve...
7,1.0,BUBLY,AD0586,1.0,"Ooh, Blackberry Bublé, my favorite. You mean B...","Ooh, Blackberry Bublé, my favorite. You mean B..."
8,1.0,Bai,AD0475,0.0,I don't want to make it tough. I just want to ...,I don't want to make it tough. I just want to ...
9,1.0,Butterfinger,AD0420,1.0,"A bull rider is bold. Now, a bull rider on a p...","A bull rider is bold. Now, a bull rider on a p..."


## Adding OCR Text

In [39]:

ocr_to_merge = pd.read_csv("./ocr_to_merge.csv")
ad_df = ad_df.merge(ocr_to_merge, left_on='commercial_number', right_on='ad', how='left')
ad_df.drop(columns=['ad', 'recognized_text'], inplace=True)
ad_df.rename(columns={'cleaned_text': 'ocr_text'}, inplace=True)

# merge ocr_text with transcript
# TODO: Rename transcript to transcript_plus_ocr
ad_df['transcript'] = ad_df['ocr_text'] + ' ' + ad_df['transcript']
ad_df.drop(columns=['ocr_text'], inplace=True)

ad_df.head()


,product_category,brand,commercial_number,BDM,transcript,audio_only_transcript
0,1.0,AvocadosfromMexico,AD0357,0.0,FIRST DRAFT EVER DRAFT SLOTH DODO BIRD FIRE PO...,"With the next pick in the first draft ever, Au..."
1,1.0,AvocadosfromMexico,AD0414,1.0,I Avocados Mexico ALWAYS IN SEASON Over here w...,Over here we have the cube of Rubik. This simp...
2,1.0,AvocadosfromMexico,AD0474,0.0,"HEALTHY Avocados Mexico ALWAYS IN SEASON Guys,...","Guys, how can we be a secret society if we can..."
3,1.0,AvocadosfromMexico,AD0525,1.0,Chris Meet Cl Won Meet Avocados Mexico ALWAYS ...,"Well, we did it. We sealed the bad out and kep..."
4,1.0,AvocadosfromMexico,AD0584,1.0,Canine Avocados From Mexico OS Me dos From Avo...,"Pageantry, poise, welcome to the dog show wher..."


In [40]:
!python -m spacy download en_core_web_sm


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Determining Frequency of Superlatives and Comparative Adjectives

In [41]:
import spacy
import pandas as pd
from collections import Counter
import text_analysis as ta

# Create new columns to store the counts and percentages
ad_df['word_count'] = 0
ad_df['superlative_count'] = 0
ad_df['comparative_count'] = 0
ad_df['uniqueness_count'] = 0
ad_df['superlative_pct'] = 0.0
ad_df['comparative_pct'] = 0.0
ad_df['uniqueness_pct'] = 0.0
ad_df['total_bdm_terms_count'] = 0
ad_df['total_bdm_terms_pct'] = 0.0

# Process each row
for idx, row in ad_df.iterrows():
    word_count = len(ta.get_tokens(row['transcript']))
    ad_df.at[idx, 'word_count'] = word_count

    superlatives = ta.get_superlatives(row['transcript'])
    ad_df.at[idx, 'superlatives'] = ', '.join(superlatives) if superlatives else ''
    superlative_count = len(superlatives) if superlatives else 0
    ad_df.at[idx, 'superlative_count'] = superlative_count

    comparatives = ta.get_comparatives(row['transcript'])
    ad_df.at[idx, 'comparatives'] = ', '.join(comparatives) if comparatives else ''
    comparative_count = len(comparatives) if comparatives else 0
    ad_df.at[idx, 'comparative_count'] = comparative_count
    
    unique_words = ta.get_unique_words(row['transcript'])
    ad_df.at[idx, 'unique_words'] = ', '.join(unique_words) if unique_words else ''
    uniqueness_count = len(unique_words) if unique_words else 0
    ad_df.at[idx, 'uniqueness_count'] = uniqueness_count

    if word_count > 0:
        ad_df.at[idx, 'superlative_pct'] = superlative_count / word_count * 100
        ad_df.at[idx, 'comparative_pct'] = comparative_count / word_count * 100
        ad_df.at[idx, 'uniqueness_pct'] = uniqueness_count / word_count * 100
        
        total_bdm_terms = superlative_count + comparative_count + uniqueness_count
        ad_df.at[idx, 'total_bdm_terms_count'] = total_bdm_terms
        ad_df.at[idx, 'total_bdm_terms_pct'] = total_bdm_terms / word_count * 100

# Sort the DataFrame
ad_df = ad_df.sort_values(
    by=['superlative_count', 'comparative_count', 'superlative_pct', 'comparative_pct', 'uniqueness_pct'],
    ascending=[False, False, False, False, False]
)

# Display top 10 results
ad_df.head(10)

,product_category,brand,commercial_number,BDM,transcript,audio_only_transcript,word_count,superlative_count,comparative_count,uniqueness_count,superlative_pct,comparative_pct,uniqueness_pct,total_bdm_terms_count,total_bdm_terms_pct,superlatives,comparatives,unique_words
393,19.0,LittleCaesars,AD0654,1.0,Delivery available from locations with online ...,Little Caesars Delivery. That's the best thing...,121,4,0,1,3.305785,0.000000,0.826446,5,4.132231,"best, best, Best, best",,only
454,21.0,Sprint,AD0617,1.0,Compared Verizon Go Unlimited and Subject cred...,How do we tell people they get the best of bot...,161,3,2,6,1.863354,1.242236,3.726708,11,6.832298,"best, best, best","More, more","Compared, Compared, Advanced, compared, compar..."
521,25.0,Google,AD0647,0.0,how to not remember that what you told me to r...,"Hey Google, show me photos of me and Loretta. ...",377,3,1,0,0.795756,0.265252,0.000000,4,1.061008,"most, luckiest, luckiest",More,
266,5.0,Valeant,AD0462,1.0,Topical Solution Toenail fungus is also known ...,"Go, go, go, go, go, go! Dirt stop! Ha, ha, ha!...",202,3,0,0,1.485149,0.000000,0.000000,3,1.485149,"Most, best, Most",,
540,25.0,TOURISMAUSTRALIA,AD0567,1.0,L M S RESTAURANTS Sponsored Dy Tourism LIKE AU...,Hey. That's not a knife. That's a knife. That'...,209,3,0,1,1.435407,0.000000,0.478469,4,1.913876,"finest, best, best",,finest
453,21.0,Sprint,AD0559,1.0,Ur Unlimited Sprint from Sprint C Unlimited Fr...,This is the fastest progress we've seen from a...,297,2,3,3,0.673401,1.010101,1.010101,8,2.693603,"latest, fastest","less, less, less","compared, Compared, Only"
275,6.0,PersilProclean,AD0612,1.0,STAIN LAB WATER V I IN Original ITS MORE THAN ...,Want to know Purcell's deepest secret? It's in...,62,2,2,2,3.225806,3.225806,3.225806,6,9.677419,"deepest, deepest","MORE, deeper","Original, exceptional"
482,21.0,VerizonWireless,AD0738,1.0,Ultra parts of more Or mobile recorded during ...,gamers he who is good for making excuses is se...,172,2,2,2,1.162791,1.162791,1.162791,6,3.488372,"fastest, best","more, more","only, only"
306,7.0,SimpliSafe,AD0615,0.0,It Cc s Signs THAN WORSE YESTERDAY WAS o Reces...,Janet saw a creepy guy in the cul-de-sac last ...,397,2,2,1,0.503778,0.503778,0.251889,5,1.259446,"best, best","less, more",compared
296,7.0,AmazonAlexa,AD0742,0.0,"S 5 Hey babe, check this out. Alexa, it's game...","Hey babe, check this out. Alexa, it's game day...",191,2,1,0,1.047120,0.523560,0.000000,3,1.570681,"worst, worst",better,


In [42]:
# remove superlatives, comparatives and unique_words from ad_df
# TODO: Comment back in
# ad_df.drop(columns=['superlatives', 'comparatives', 'unique_words', 'bdm_words'], inplace=True)
ad_df.drop(columns=['comparative_pct', 'superlative_pct', 'uniqueness_pct'], inplace=True)
ad_df.head(10)


,product_category,brand,commercial_number,BDM,transcript,audio_only_transcript,word_count,superlative_count,comparative_count,uniqueness_count,total_bdm_terms_count,total_bdm_terms_pct,superlatives,comparatives,unique_words
393,19.0,LittleCaesars,AD0654,1.0,Delivery available from locations with online ...,Little Caesars Delivery. That's the best thing...,121,4,0,1,5,4.132231,"best, best, Best, best",,only
454,21.0,Sprint,AD0617,1.0,Compared Verizon Go Unlimited and Subject cred...,How do we tell people they get the best of bot...,161,3,2,6,11,6.832298,"best, best, best","More, more","Compared, Compared, Advanced, compared, compar..."
521,25.0,Google,AD0647,0.0,how to not remember that what you told me to r...,"Hey Google, show me photos of me and Loretta. ...",377,3,1,0,4,1.061008,"most, luckiest, luckiest",More,
266,5.0,Valeant,AD0462,1.0,Topical Solution Toenail fungus is also known ...,"Go, go, go, go, go, go! Dirt stop! Ha, ha, ha!...",202,3,0,0,3,1.485149,"Most, best, Most",,
540,25.0,TOURISMAUSTRALIA,AD0567,1.0,L M S RESTAURANTS Sponsored Dy Tourism LIKE AU...,Hey. That's not a knife. That's a knife. That'...,209,3,0,1,4,1.913876,"finest, best, best",,finest
453,21.0,Sprint,AD0559,1.0,Ur Unlimited Sprint from Sprint C Unlimited Fr...,This is the fastest progress we've seen from a...,297,2,3,3,8,2.693603,"latest, fastest","less, less, less","compared, Compared, Only"
275,6.0,PersilProclean,AD0612,1.0,STAIN LAB WATER V I IN Original ITS MORE THAN ...,Want to know Purcell's deepest secret? It's in...,62,2,2,2,6,9.677419,"deepest, deepest","MORE, deeper","Original, exceptional"
482,21.0,VerizonWireless,AD0738,1.0,Ultra parts of more Or mobile recorded during ...,gamers he who is good for making excuses is se...,172,2,2,2,6,3.488372,"fastest, best","more, more","only, only"
306,7.0,SimpliSafe,AD0615,0.0,It Cc s Signs THAN WORSE YESTERDAY WAS o Reces...,Janet saw a creepy guy in the cul-de-sac last ...,397,2,2,1,5,1.259446,"best, best","less, more",compared
296,7.0,AmazonAlexa,AD0742,0.0,"S 5 Hey babe, check this out. Alexa, it's game...","Hey babe, check this out. Alexa, it's game day...",191,2,1,0,3,1.570681,"worst, worst",better,


# Product Category DF

In [43]:

product_brands_df = pd.read_csv("product_categories.csv")
product_brands_df.head(40)
product_brands_df = product_brands_df.drop('product_cat_id', axis=1)
ad_df = ad_df.drop('product_category', axis=1)
display(product_brands_df)
display(ad_df)

,product_cat_name,product_cat_keywords,product_cat_brands
0,Alcoholic beverages (Beer Hard Seltzer),"['smooth', 'rich', 'refreshing', 'aromatic', '...","['Anheuser Busch InBev', 'Becks Beer', 'Bud Li..."
1,Banking & Investments,"['secure', 'reliable', 'customized', 'personal...","['BankofAmerica', 'Coinbase', 'Cryptocom', 'Di..."
2,Car Accessories & Related Services,"['safe', 'protect', 'protected', 'protection',...","['Michelin', 'Wallbox', 'Weathertech']"
3,Car Manufacturer,"['Luxurious', 'Efficient', 'Powerful', 'Innova...","['Acura', 'AlfaRomeo', 'Audi', 'BMW', 'Buick',..."
4,Car Sales & Services Platform,"['easy', 'perfect', 'love', 'expert', 'really'...","['Carvana', 'Vroom', 'Carscom', 'Carmax']"
5,"Clothing, shoes and apparel","['favorite', 'designed', 'comfortable', 'uncom...","['CalvinKlein', 'Gildan', 'HANDM', 'Marmot', '..."
6,Consumer Electronics and Appliances,"['Smart', 'Display', 'Control', 'Touchscreen',...","['BlackBerry', 'Google', 'Intel', 'LGEEAudioVi..."
7,Cosmetics and personal care products,"['luxurious', 'nourishing', 'radiant', 'smooth...","['Fitbit', 'PlanetFitness']"
8,Diet and exercise products,"['healthy', 'energizing', 'effective', 'smart'...","['CalvinKlein', 'Gildan', 'Marmot', 'Skechers'..."
9,Food Delivery Services,"['easy', 'delivered', 'local', 'convenience', ...","['DoorDash', 'UberEats']"


,brand,commercial_number,BDM,transcript,audio_only_transcript,word_count,superlative_count,comparative_count,uniqueness_count,total_bdm_terms_count,total_bdm_terms_pct,superlatives,comparatives,unique_words
393,LittleCaesars,AD0654,1.0,Delivery available from locations with online ...,Little Caesars Delivery. That's the best thing...,121,4,0,1,5,4.132231,"best, best, Best, best",,only
454,Sprint,AD0617,1.0,Compared Verizon Go Unlimited and Subject cred...,How do we tell people they get the best of bot...,161,3,2,6,11,6.832298,"best, best, best","More, more","Compared, Compared, Advanced, compared, compar..."
521,Google,AD0647,0.0,how to not remember that what you told me to r...,"Hey Google, show me photos of me and Loretta. ...",377,3,1,0,4,1.061008,"most, luckiest, luckiest",More,
266,Valeant,AD0462,1.0,Topical Solution Toenail fungus is also known ...,"Go, go, go, go, go, go! Dirt stop! Ha, ha, ha!...",202,3,0,0,3,1.485149,"Most, best, Most",,
540,TOURISMAUSTRALIA,AD0567,1.0,L M S RESTAURANTS Sponsored Dy Tourism LIKE AU...,Hey. That's not a knife. That's a knife. That'...,209,3,0,1,4,1.913876,"finest, best, best",,finest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Squarespace,AD0455,1.0,REAL REAL TALK Real TALK TALK Real REAL TALK F...,S-U-C-C-E-S-S-E to the S. Successes. Our entir...,152,0,0,0,0,0.000000,,,
535,Squarespace,AD0508,0.0,G ET U R D O MA N B E F 0 RE 6 0 N E MAKE YOUR...,"No, it's not a joke. Listen, I'm trying to sta...",104,0,0,0,0,0.000000,,,
537,Squarespace,AD0675,0.0,Welcome MINNESOTA Est To A website makes it re...,"Excuse me, ma'am. What you doing over there? I...",66,0,0,0,0,0.000000,,,
538,Squarespace,AD0730,0.0,1 1 7 6 delete Job caps MOVE HOME ANEW TAKE ON...,"Working 5 to 9, you've got passion and a visio...",74,0,0,0,0,0.000000,,,


In [44]:
# Create a dictionary to map brands to their product categories and other attributes
brand_to_info = {}
for _, row in product_brands_df.iterrows():
    # Convert string representation of list to actual list
    brands = eval(row['product_cat_brands'])
    for brand in brands:
        # Remove spaces and convert to lowercase for more robust matching
        brand = brand.replace(' ', '').lower()
        # Store all columns for this brand
        brand_to_info[brand] = {col: row[col] for col in product_brands_df.columns}

# Function to find category info for a brand
def find_brand_info(brand):
    # Clean brand name for matching
    clean_brand = brand.replace(' ', '').lower()
    return brand_to_info.get(clean_brand)

# Add all product category columns to ad_df
for col in product_brands_df.columns:
    ad_df[col] = ad_df['brand'].apply(lambda x: find_brand_info(x)[col] if find_brand_info(x) else None)

# Print brands that couldn't be mapped
unmapped_brands = ad_df[ad_df['product_cat_name'].isna()]['brand'].unique()
if len(unmapped_brands) > 0:
    print("Brands without category mapping:")
    for brand in unmapped_brands:
        print(f"- {brand}")

# Print number of rows with missing category
print(f"Number of rows where product category is NaN: {ad_df[ad_df['product_cat_name'].isna()].shape[0]}")

# Drop rows with missing categories
ad_df = ad_df.dropna(subset=['product_cat_name'])

print(f"Final number of rows with missing categories: {ad_df[ad_df['product_cat_name'].isna()].shape[0]}")

Brands without category mapping:
- Volkswagen
- Beck's Beer
- AvocadosfromMexico
- THEWASHINGTONPOST
- BassProShops
- Colgate
- Century21
- Schick
- DoveMenCare
- Hyndai
- IrishSpring
- DrSquatch
- Groupon
- CaesarsSportsbook
- LGCEAudioVisual
- Always
- Axe
- DollarShaveClub
- Gillette
- Huggies
- Itsa10Haircare
- Olay
- Weightwatchers
- UNIVERSALPARKSANDRESORTS
- Mercari
- Fiverr
- Indeed
Number of rows where product category is NaN: 38
Final number of rows with missing categories: 0


In [45]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict

nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/nilst/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/nilst/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/nilst/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/nilst/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/nilst/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_dat

True

In [46]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')
def get_semantic_similarity(text, keywords):
    # Convert string keywords list to actual list if needed
    if isinstance(keywords, str):
        keywords = eval(keywords)
    
    # Combine keywords into a single string
    keywords_text = ' '.join(keywords)
    
    # Get embeddings
    text_embedding = model.encode([str(text)])[0]
    keywords_embedding = model.encode([keywords_text])[0]
    
    # Calculate cosine similarity
    similarity = np.dot(text_embedding, keywords_embedding) / (
        np.linalg.norm(text_embedding) * np.linalg.norm(keywords_embedding)
    )
    
    # Find most similar phrases
    sentences = str(text).split('.')
    sentence_embeddings = model.encode(sentences)
    keyword_embedding = model.encode([keywords_text])[0]
    
    # Calculate similarities for each sentence
    sentence_similarities = np.dot(sentence_embeddings, keyword_embedding) / (
        np.linalg.norm(sentence_embeddings, axis=1) * 
        np.linalg.norm(keyword_embedding)
    )
    
    # Get top 3 most similar sentences
    top_indices = np.argsort(sentence_similarities)[-3:][::-1]
    similar_phrases = [sentences[i].strip() for i in top_indices if sentence_similarities[i] > 0.3]
    
    return similarity, similar_phrases

# Apply the analysis to the DataFrame
similarities = []
similar_phrases = []

for _, row in ad_df.iterrows():
    sim, phrases = get_semantic_similarity(row['transcript'], row['product_cat_keywords'])
    similarities.append(sim)
    similar_phrases.append(phrases)

# Add new columns to DataFrame
ad_df['product_cat_keyword_similarity'] = similarities
ad_df['similar_phrases'] = similar_phrases


ad_df.head(10)

,brand,commercial_number,BDM,transcript,audio_only_transcript,word_count,superlative_count,comparative_count,uniqueness_count,total_bdm_terms_count,total_bdm_terms_pct,superlatives,comparatives,unique_words,product_cat_name,product_cat_keywords,product_cat_brands,product_cat_keyword_similarity,similar_phrases
393,LittleCaesars,AD0654,1.0,Delivery available from locations with online ...,Little Caesars Delivery. That's the best thing...,121,4,0,1,5,4.132231,"best, best, Best, best",,only,Quick Service Restaurants & Food Chains,"['fresh', 'enjoy', 'organic', 'natural', 'deli...","['BurgerKing', 'Chipotle', 'HardRock', 'JimmyJ...",0.223981,[]
454,Sprint,AD0617,1.0,Compared Verizon Go Unlimited and Subject cred...,How do we tell people they get the best of bot...,161,3,2,6,11,6.832298,"best, best, best","More, more","Compared, Compared, Advanced, compared, compar...",Telecommuniactions services,"['5G', 'LTE', 'network', 'sound', 'unlimited',...","['Mint', 'Sprint', 'TMobile', 'VerizonWireless']",0.423689,[And a bird horse! Why don't you just tell peo...
521,Google,AD0647,0.0,how to not remember that what you told me to r...,"Hey Google, show me photos of me and Loretta. ...",377,3,1,0,4,1.061008,"most, luckiest, luckiest",More,,Smart Home Devices,"['Safe', 'Trusted', 'Reliable', 'Efficient', '...","['ADT', 'Amazon', 'AmazonAlexa', 'AmazonEcho',...",0.097371,[]
266,Valeant,AD0462,1.0,Topical Solution Toenail fungus is also known ...,"Go, go, go, go, go, go! Dirt stop! Ha, ha, ha!...",202,3,0,0,3,1.485149,"Most, best, Most",,,Healthcare and Medical Products,"['reliable', 'effective', 'innovative', 'advan...","['CueHealth', 'Dexcom', 'Hologic', 'Advil', 'A...",0.065155,[]
540,TOURISMAUSTRALIA,AD0567,1.0,L M S RESTAURANTS Sponsored Dy Tourism LIKE AU...,Hey. That's not a knife. That's a knife. That'...,209,3,0,1,4,1.913876,"finest, best, best",,finest,Travel and Booking Platforms,"['Discover', 'Widen', 'Feel', 'Connect', 'Go',...","['Airbnb', 'Apartmentscom', 'TOURISMAUSTRALIA'...",0.104097,[]
453,Sprint,AD0559,1.0,Ur Unlimited Sprint from Sprint C Unlimited Fr...,This is the fastest progress we've seen from a...,297,2,3,3,8,2.693603,"latest, fastest","less, less, less","compared, Compared, Only",Telecommuniactions services,"['5G', 'LTE', 'network', 'sound', 'unlimited',...","['Mint', 'Sprint', 'TMobile', 'VerizonWireless']",0.405159,[Ur Unlimited Sprint from Sprint C Unlimited F...
275,PersilProclean,AD0612,1.0,STAIN LAB WATER V I IN Original ITS MORE THAN ...,Want to know Purcell's deepest secret? It's in...,62,2,2,2,6,9.677419,"deepest, deepest","MORE, deeper","Original, exceptional",Household and lawn and garden supplies,"['powerful', 'effective', 'reliable', 'fast-ac...","['Febreze', 'Loctite', 'MrClean', 'PersilProCl...",0.370549,"[Purcell Pro Clean, STAIN LAB WATER V I IN Ori..."
482,VerizonWireless,AD0738,1.0,Ultra parts of more Or mobile recorded during ...,gamers he who is good for making excuses is se...,172,2,2,2,6,3.488372,"fastest, best","more, more","only, only",Telecommuniactions services,"['5G', 'LTE', 'network', 'sound', 'unlimited',...","['Mint', 'Sprint', 'TMobile', 'VerizonWireless']",0.476194,[Ultra parts of more Or mobile recorded during...
306,SimpliSafe,AD0615,0.0,It Cc s Signs THAN WORSE YESTERDAY WAS o Reces...,Janet saw a creepy guy in the cul-de-sac last ...,397,2,2,1,5,1.259446,"best, best","less, more",compared,Smart Home Devices,"['Safe', 'Trusted', 'Reliable', 'Efficient', '...","['ADT', 'Amazon', 'AmazonAlexa', 'AmazonEcho',...",0.346275,[With an arsenal of sensors designed to blanke...
296,AmazonAlexa,AD0742,0.0,"S 5 Hey babe, check this out. Alexa, it's game...","Hey babe, check this out. Alexa, it's game day...",191,2,1,0,3,1.570681,"worst, worst",better,,Smart Home Devices,"['Safe', 'Trusted', 'Reliable', 'Efficient', '...","['ADT', 'Amazon', 'AmazonAlexa', 'AmazonEcho',...",0.129642,[]


In [47]:
# TODO: Implement proper handling of missing values

ad_df[ad_df.isnull().any(axis=1)].head()
ad_df[ad_df.isna().any(axis=1)].head()

ad_df = ad_df.dropna()
ad_df = ad_df[ad_df['transcript'] != '']
ad_df = ad_df[ad_df['transcript'] != '']

# print all from ad_df with empty values
print(ad_df[ad_df.isnull().any(axis=1)])
print(ad_df[ad_df.isna().any(axis=1)])

Empty DataFrame
Columns: [brand, commercial_number, BDM, transcript, audio_only_transcript, word_count, superlative_count, comparative_count, uniqueness_count, total_bdm_terms_count, total_bdm_terms_pct, superlatives, comparatives, unique_words, product_cat_name, product_cat_keywords, product_cat_brands, product_cat_keyword_similarity, similar_phrases]
Index: []
Empty DataFrame
Columns: [brand, commercial_number, BDM, transcript, audio_only_transcript, word_count, superlative_count, comparative_count, uniqueness_count, total_bdm_terms_count, total_bdm_terms_pct, superlatives, comparatives, unique_words, product_cat_name, product_cat_keywords, product_cat_brands, product_cat_keyword_similarity, similar_phrases]
Index: []


In [48]:
ad_df.head(20)

,brand,commercial_number,BDM,transcript,audio_only_transcript,word_count,superlative_count,comparative_count,uniqueness_count,total_bdm_terms_count,total_bdm_terms_pct,superlatives,comparatives,unique_words,product_cat_name,product_cat_keywords,product_cat_brands,product_cat_keyword_similarity,similar_phrases
393,LittleCaesars,AD0654,1.0,Delivery available from locations with online ...,Little Caesars Delivery. That's the best thing...,121,4,0,1,5,4.132231,"best, best, Best, best",,only,Quick Service Restaurants & Food Chains,"['fresh', 'enjoy', 'organic', 'natural', 'deli...","['BurgerKing', 'Chipotle', 'HardRock', 'JimmyJ...",0.223981,[]
454,Sprint,AD0617,1.0,Compared Verizon Go Unlimited and Subject cred...,How do we tell people they get the best of bot...,161,3,2,6,11,6.832298,"best, best, best","More, more","Compared, Compared, Advanced, compared, compar...",Telecommuniactions services,"['5G', 'LTE', 'network', 'sound', 'unlimited',...","['Mint', 'Sprint', 'TMobile', 'VerizonWireless']",0.423689,[And a bird horse! Why don't you just tell peo...
521,Google,AD0647,0.0,how to not remember that what you told me to r...,"Hey Google, show me photos of me and Loretta. ...",377,3,1,0,4,1.061008,"most, luckiest, luckiest",More,,Smart Home Devices,"['Safe', 'Trusted', 'Reliable', 'Efficient', '...","['ADT', 'Amazon', 'AmazonAlexa', 'AmazonEcho',...",0.097371,[]
266,Valeant,AD0462,1.0,Topical Solution Toenail fungus is also known ...,"Go, go, go, go, go, go! Dirt stop! Ha, ha, ha!...",202,3,0,0,3,1.485149,"Most, best, Most",,,Healthcare and Medical Products,"['reliable', 'effective', 'innovative', 'advan...","['CueHealth', 'Dexcom', 'Hologic', 'Advil', 'A...",0.065155,[]
540,TOURISMAUSTRALIA,AD0567,1.0,L M S RESTAURANTS Sponsored Dy Tourism LIKE AU...,Hey. That's not a knife. That's a knife. That'...,209,3,0,1,4,1.913876,"finest, best, best",,finest,Travel and Booking Platforms,"['Discover', 'Widen', 'Feel', 'Connect', 'Go',...","['Airbnb', 'Apartmentscom', 'TOURISMAUSTRALIA'...",0.104097,[]
453,Sprint,AD0559,1.0,Ur Unlimited Sprint from Sprint C Unlimited Fr...,This is the fastest progress we've seen from a...,297,2,3,3,8,2.693603,"latest, fastest","less, less, less","compared, Compared, Only",Telecommuniactions services,"['5G', 'LTE', 'network', 'sound', 'unlimited',...","['Mint', 'Sprint', 'TMobile', 'VerizonWireless']",0.405159,[Ur Unlimited Sprint from Sprint C Unlimited F...
275,PersilProclean,AD0612,1.0,STAIN LAB WATER V I IN Original ITS MORE THAN ...,Want to know Purcell's deepest secret? It's in...,62,2,2,2,6,9.677419,"deepest, deepest","MORE, deeper","Original, exceptional",Household and lawn and garden supplies,"['powerful', 'effective', 'reliable', 'fast-ac...","['Febreze', 'Loctite', 'MrClean', 'PersilProCl...",0.370549,"[Purcell Pro Clean, STAIN LAB WATER V I IN Ori..."
482,VerizonWireless,AD0738,1.0,Ultra parts of more Or mobile recorded during ...,gamers he who is good for making excuses is se...,172,2,2,2,6,3.488372,"fastest, best","more, more","only, only",Telecommuniactions services,"['5G', 'LTE', 'network', 'sound', 'unlimited',...","['Mint', 'Sprint', 'TMobile', 'VerizonWireless']",0.476194,[Ultra parts of more Or mobile recorded during...
306,SimpliSafe,AD0615,0.0,It Cc s Signs THAN WORSE YESTERDAY WAS o Reces...,Janet saw a creepy guy in the cul-de-sac last ...,397,2,2,1,5,1.259446,"best, best","less, more",compared,Smart Home Devices,"['Safe', 'Trusted', 'Reliable', 'Efficient', '...","['ADT', 'Amazon', 'AmazonAlexa', 'AmazonEcho',...",0.346275,[With an arsenal of sensors designed to blanke...
296,AmazonAlexa,AD0742,0.0,"S 5 Hey babe, check this out. Alexa, it's game...","Hey babe, check this out. Alexa, it's game day...",191,2,1,0,3,1.570681,"worst, worst",better,,Smart Home Devices,"['Safe', 'Trusted', 'Reliable', 'Efficient', '...","['ADT', 'Amazon', 'AmazonAlexa', 'AmazonEcho',...",0.129642,[]


## Nomen + Adjektive

In [49]:
def extract_adj_noun_pairs(text):
    doc = nlp(text)
    pairs = []
    for token in doc:
        # Check if the token is an adjective (ADJ)
        if token.pos_ == "ADJ":
            # Ensure the token is not the last one in the document
            if token.i < len(doc) - 1:  
                # Check if the next token is a noun (NOUN) and directly follows the adjective
                next_token = token.nbor(1)
                if next_token.pos_ == "NOUN":
                    pairs.append(f"{token.text} {next_token.text}")  # "adjective noun"
    return pairs

# Apply the function to the transcript column
ad_df["adj_noun_pairs"] = ad_df["transcript"].apply(extract_adj_noun_pairs)

# Add a column for the number of adjective-noun pairs
ad_df["num_adj_noun_pairs"] = ad_df["adj_noun_pairs"].apply(len)

# Display the first 10 rows of the DataFrame
display(ad_df.head(10))

NameError: name 'nlp' is not defined

In [ ]:

# show all rows ehere bdm is 1.0 and where industry is product_cat_id 4

nice_df = ad_df[ad_df['BDM'] == 1.0]


# Create a function to find shared words
def get_shared_words(row):
    # Convert keywords string to list if it's a string
    keywords = eval(row['product_cat_keywords']) if isinstance(row['product_cat_keywords'], str) else row['product_cat_keywords']
    
    # Convert all keywords to lowercase for better matching
    keywords = [word.lower() for word in keywords]
    
    # Split transcript into words and convert to lowercase
    transcript_words = set(word.lower() for word in str(row['transcript']).split())
    
    # Find intersection between keywords and transcript words
    shared = [word for word in keywords if word in transcript_words]
    
    return shared

# Add new column for shared words
ad_df['shared_keywords'] = ad_df.apply(get_shared_words, axis=1)

# Add column for count of shared words
ad_df['shared_keywords_count'] = ad_df['shared_keywords'].str.len()

# Update the display code
nice_df = ad_df[ad_df['BDM'] == 1.0]
display(nice_df)

In [ ]:
# Calculate the minimum number of samples in each group
# 
min_samples = min(len(ad_df[ad_df['BDM'] == 1]), len(ad_df[ad_df['BDM'] == 0]))
ad_df.to_csv('ad_df.csv', index=False)
# Perform undersampling
ad_df_balanced = pd.concat([
    ad_df[ad_df['BDM'] == 1].sample(n=min_samples, random_state=42),
    ad_df[ad_df['BDM'] == 0].sample(n=min_samples, random_state=42)
]).reset_index(drop=True)

# Print the results
print(f"Total rows: {len(ad_df_balanced)}")
print(f"Rows with BDM = 1.0: {len(ad_df_balanced[ad_df_balanced['BDM'] == 1.0])}")
print(f"Rows with BDM = 0.0: {len(ad_df_balanced[ad_df_balanced['BDM'] == 0.0])}")



commercial_numbers = ad_df_balanced['commercial_number']


ad_df_balanced.head(20)

## Ansatz 1 (Machine learning)

In [30]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
# Prepare features
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown
from sklearn.metrics import confusion_matrix, roc_curve, auc
# import gridsearchcv
from sklearn.model_selection import GridSearchCV




In [31]:
# Prepare the data
data = ad_df_balanced[['product_cat_keyword_similarity', 'total_bdm_terms_pct', 'num_adj_noun_pairs', 'total_bdm_terms_count']]
target = ad_df_balanced['BDM']

In [32]:
def get_base_models():
    """Return dictionary of base model configurations"""
    return {
        'Majority Classifier': DummyClassifier(),
        'Logistic Regression': LogisticRegression(random_state=42),
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'Support Vector Machine': SVC(random_state=42),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'AdaBoost': AdaBoostClassifier(random_state=42),
    }

In [ ]:



from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

def get_param_distributions():
    """Return parameter distributions for RandomizedSearchCV"""
    return {
        'Logistic Regression': {
            'C': uniform(0.1, 10.0),
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear']
        },
        'Decision Tree': {
            'max_depth': randint(3, 10),
            'min_samples_split': randint(2, 20),
            'min_samples_leaf': randint(1, 10)
        },
        'Random Forest': {
            'n_estimators': randint(50, 300),
            'max_depth': randint(3, 10),
            'min_samples_split': randint(2, 20)
        },
        'Support Vector Machine': {
            'C': uniform(0.1, 20.0),
            'kernel': ['rbf', 'linear']
        },
        'Gradient Boosting': {
            'n_estimators': randint(50, 300),
            'learning_rate': uniform(0.01, 0.3),
            'max_depth': randint(3, 10)
        },
        'Dummy Classifier': {
            'strategy': ['stratified']
        }
    }

def tune_models(X, y, models, param_distributions, cv=5, n_iter=20):
    """Perform RandomizedSearchCV on specified models"""
    tuned_models = {}
    
    for name, model in models.items():
        if name in param_distributions:
            print(f"\nTuning {name}...")
            random_search = RandomizedSearchCV(
                model,
                param_distributions[name],
                n_iter=n_iter,
                cv=cv,
                scoring='roc_auc',
                n_jobs=-1,
                random_state=42
            )
            random_search.fit(X, y)
            tuned_models[name] = random_search.best_estimator_
            print(f"Best parameters: {random_search.best_params_}")
            print(f"Best score: {random_search.best_score_:.3f}")
        else:
            tuned_models[name] = model
            
    return tuned_models

# Usage:
base_models = get_base_models()
param_distributions = get_param_distributions()
models = tune_models(data, target, base_models, param_distributions)

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.metrics import (
    roc_auc_score, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    confusion_matrix,
    classification_report
)



In [ ]:
def evaluate_models(X, y, models, cv=5):
    """
    Evaluate models and add predictions to original dataframe
    """
    results = []
    predictions = pd.DataFrame()
    
    for name, model in models.items():
        print(f"Evaluating {name}...")
        model.fit(X, y)
        
        # Get predictions
        y_pred = model.predict(X)
        
        # Create confusion matrix categories
        pred_categories = []
        for true, pred in zip(y, y_pred):
            if true == 0 and pred == 0:
                pred_categories.append('TN')
            elif true == 0 and pred == 1:
                pred_categories.append('FP')
            elif true == 1 and pred == 0:
                pred_categories.append('FN')
            else:  # true == 1 and pred == 1
                pred_categories.append('TP')
        
        predictions[f'{name}_result'] = pred_categories
        
        # Perform cross-validation
        cv_results = cross_validate(
            model, 
            X, 
            y, 
            cv=cv, 
            scoring=['roc_auc', 'accuracy', 'precision', 'recall']
        )
        
        # Calculate metrics as before
        results.append({
            'Model': name,
            'ROC AUC (Mean)': np.mean(cv_results['test_roc_auc']),
            'ROC AUC (Std)': np.std(cv_results['test_roc_auc']),
            'Accuracy (Mean)': np.mean(cv_results['test_accuracy']),
            'Accuracy (Std)': np.std(cv_results['test_accuracy']),
            'Precision (Mean)': np.mean(cv_results['test_precision']),
            'Precision (Std)': np.std(cv_results['test_precision']),
            'Recall (Mean)': np.mean(cv_results['test_recall']),
            'Recall (Std)': np.std(cv_results['test_recall'])
        })
    
    results_df = pd.DataFrame(results).sort_values('ROC AUC (Mean)', ascending=False)
    return results_df, predictions


def plot_confusion_matrices(X, y, models):
    """
    Plot confusion matrices for each model
    
    Parameters:
    - X: Feature matrix
    - y: Target variable
    - models: Dictionary of tuned models
    """
    n_models = len(models)
    fig, axes = plt.subplots(
        nrows=(n_models + 1) // 2, 
        ncols=2, 
        figsize=(16, 4 * ((n_models + 1) // 2))
    )
    axes = axes.ravel()  # Flatten axes array
    
    for i, (name, model) in enumerate(models.items()):
        # Predict using the model
        y_pred = model.predict(X)
        
        # Compute confusion matrix
        cm = confusion_matrix(y, y_pred)
        
        # Plot confusion matrix
        sns.heatmap(
            cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues', 
            ax=axes[i]
        )
        axes[i].set_title(f'{name} Confusion Matrix')
        axes[i].set_xlabel('Predicted Label')
        axes[i].set_ylabel('True Label')
    
    # Remove extra subplots if any
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()



results_df, predictions = evaluate_models(data, target, models)
original_data = ad_df_balanced.copy()
original_data = pd.concat([original_data, commercial_numbers], axis=1)

#
# 
#  Add predictions to original data
original_data = pd.concat([original_data, predictions], axis=1)

# Display results
print("Cross-Validation Results:\n")
display(Markdown(results_df.to_markdown(index=False)))
plot_confusion_matrices(data, target, models)

# Display sample of data with predictions


# rename frame
predicted_data = original_data

# only include the top 3 models prediction results
predicted_data = predicted_data[['commercial_number', 'BDM', 'Logistic Regression_result', 'Random Forest_result', 'Support Vector Machine_result']]
# write the majority result of the colums logistic regression, random forest and support vector machine to a new column majority vote
predicted_data['majority_vote'] = predicted_data[['Logistic Regression_result', 'Random Forest_result', 'Support Vector Machine_result']].mode(axis=1)[0]
display(predicted_data.head(10))


linlu_table = predicted_data[['commercial_number', 'BDM', 'majority_vote']]
# only show those rows where the majority vote is FP or FN
linlu_table = linlu_table[(linlu_table['majority_vote'] == 'FP') | (linlu_table['majority_vote'] == 'FN')]
linlu_table['comment'] = ''
display(linlu_table.head(10))

# output to excel
linlu_table.to_excel('linlu_table.xlsx', index=False)

In [ ]:
from sklearn.tree import plot_tree

print("\nDecision Tree Analysis:")
simple_decision_tree = models['Decision Tree']
simple_decision_tree.fit(data, target)

# Create a new figure for the decision tree
plt.figure(figsize=(20, 10))
plot_tree(simple_decision_tree, 
          filled=True, 
          feature_names=data.columns, 
          class_names=['No BDM', 'BDM'])
plt.title("Simple Decision Tree Visualization")
plt.show()

# Print feature importance ranking
importances = simple_decision_tree.feature_importances_
indices = np.argsort(importances)[::-1]

print("\nFeature Importance Ranking:")
for f in range(data.shape[1]):
    print(f"{f + 1}. {data.columns[indices[f]]}: {importances[indices[f]]:.3f}")

## Ansatz 2 - RNN + LSTM

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

## Ansatz 2 - RNN + LSTM

In [38]:
df = pd.read_csv('df.csv')

In [ ]:
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])

In [40]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('label')
    
    # Clean the descriptions - replace NaN with empty string and ensure all items are strings
    descriptions = df["description"].fillna("").astype(str).tolist()
    
    # Convert to tensors
    ds = tf.data.Dataset.from_tensor_slices((descriptions, labels))
    
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [41]:
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)

# Embedding + Model

In [42]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, dtype=tf.string, trainable=True)

In [ ]:
hub_layer(list(train_data)[0][0])

In [44]:
import tf_keras

model = tf_keras.Sequential([
    hub_layer,
    tf_keras.layers.Dense(16, activation='relu'),
    tf_keras.layers.Dropout(0.4),
    tf_keras.layers.Dense(16, activation='relu'),
    tf_keras.layers.Dropout(0.4),
    tf_keras.layers.Dense(1, activation='sigmoid')
])

In [45]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.evaluate(train_data)

In [ ]:
model.evaluate(valid_data)

In [48]:
# history = model.fit(train_data, epochs=3, validation_data=valid_data)

In [49]:
# 


In [ ]:
model.evaluate(test_data)

In [51]:
from sklearn.model_selection import KFold

def cross_validate_confusion_matrix(df, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    all_true = []
    all_pred = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        # Split the data
        train = df.iloc[train_idx]
        val = df.iloc[val_idx]
        
        # Create datasets
        train_data = df_to_dataset(train)
        val_data = df_to_dataset(val)
        
        # Train model
        model = tf_keras.Sequential([
            hub_layer,
            tf_keras.layers.Dense(16, activation='relu'),
            tf_keras.layers.Dropout(0.4),
            tf_keras.layers.Dense(16, activation='relu'),
            tf_keras.layers.Dropout(0.4),
            tf_keras.layers.Dense(1, activation='sigmoid')
        ])
        
        model.compile(optimizer='adam',
                     loss=tf.keras.losses.BinaryCrossentropy(),
                     metrics=['accuracy'])
        
        model.fit(train_data, epochs=3, verbose=0)
        
        # Get predictions for this fold's validation set
        val_predictions = model.predict(val_data, verbose=0)
        val_predictions = (val_predictions > 0.5).astype(int)
        
        # Store true labels and predictions
        val_labels = np.concatenate([y for x, y in val_data], axis=0)
        all_true.extend(val_labels)
        all_pred.extend(val_predictions)
    
    return np.array(all_true), np.array(all_pred)

In [52]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(true_labels, predictions, title="Model Confusion Matrix"):
    # Create confusion matrix
    cm = confusion_matrix(true_labels, predictions)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    # Calculate and print metrics
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"\nModel Metrics:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")

In [ ]:
# Get cross-validation results
true_labels, predictions = cross_validate_confusion_matrix(df)

# Plot confusion matrix for all data
plot_confusion_matrix(true_labels, predictions, "Cross-Validated Model Confusion Matrix")

# LSTM

In [54]:
def cross_validate_lstm_confusion_matrix(df, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    all_true = []
    all_pred = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        # Split the data
        train = df.iloc[train_idx]
        val = df.iloc[val_idx]
        
        # Create datasets
        train_data = df_to_dataset(train)
        val_data = df_to_dataset(val)
        
        # Create and compile model
        model = tf.keras.Sequential([
            encoder,
            tf.keras.layers.Embedding(
                input_dim=len(encoder.get_vocabulary()),
                output_dim=32,
                mask_zero=True
            ),
            tf.keras.layers.LSTM(32, use_cudnn=False),  # Disable cuDNN
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss=tf.keras.losses.BinaryCrossentropy(),
                     metrics=['accuracy'])
        
        # Train model
        model.fit(train_data, epochs=5, verbose=0)
        
        # Get predictions for this fold's validation set
        val_predictions = model.predict(val_data, verbose=0)
        val_predictions = (val_predictions > 0.5).astype(int)
        
        # Store true labels and predictions
        val_labels = np.concatenate([y for x, y in val_data], axis=0)
        all_true.extend(val_labels)
        all_pred.extend(val_predictions)
    
    return np.array(all_true), np.array(all_pred)

In [55]:
encoder = tf.keras.layers.TextVectorization(max_tokens=600)
encoder.adapt(train_data.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

In [57]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=32,
        mask_zero=True
    ),
    tf.keras.layers.LSTM(32, use_cudnn=False),  # Disable cuDNN
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [58]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.evaluate(train_data)
model.evaluate(valid_data)

In [ ]:
history = model.fit(train_data, epochs=5, validation_data=valid_data)

In [ ]:
model.evaluate(test_data)

In [ ]:
# Get cross-validation results for LSTM
true_labels_lstm, predictions_lstm = cross_validate_lstm_confusion_matrix(df)

# Plot confusion matrix for all data
plot_confusion_matrix(true_labels_lstm, predictions_lstm, "Cross-Validated LSTM Model Confusion Matrix")

<div class="alert alert-block alert-info">
<b>Fazit</b>


Warum performen die ML Modelle besser als die Neural Networks

1. Datensatzgröße:
- 250 Werbespots sind ein sehr kleiner Datensatz für neuronale Netze, die typischerweise Tausende oder Zehntausende von Beispielen benötigen, um effektiv zu lernen


- Feature-Extraktionsansatz bietet eine explizite Struktur, die bei der begrenzten Menge schonmal eine Grundlage schaffen, die die NNs nicht haben

2. Featurequalität vs. Rohtext:
- Unsere handgefertigten Features erfassen domänenspezifisches Wissen darüber, was eine BDM effektiv macht

- Neuronale Netze müssen diese Muster von Grund auf nur aus Rohtext lernen, was mit begrenzten Daten viel schwieriger ist

</div>